Every Corpus has three main components: Conversations, Utterances, and Speakers.

See this link for more in depth tutorial: https://convokit.cornell.edu/documentation/tutorial.html

In [1]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import torch 
from torch.utils.data import Dataset
from transformers import BertTokenizer, AutoTokenizer
from tqdm import tqdm
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.classes.preprocessor import TextPreProcessor
from model1.spanemo.data_loader import DataClass
from fastprogress.fastprogress import format_time, master_bar, progress_bar
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import datetime
import numpy as np
import json

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
seed = 12345678

if str(device) == 'cuda:0':
    print("Currently using GPU: {}".format(device))
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    print("WARNING: USING CPU")

Currently using GPU: cuda:0


## Download Data

In [3]:
# from convokit import Corpus, download
# corpus = Corpus(filename=download("movie-corpus"))

In [4]:
# corpus.print_summary_stats()

In [5]:
# speakers_df = corpus.get_speakers_dataframe()

In [6]:
# speakers_df.head()  

In [7]:
# convo_df = corpus.get_conversations_dataframe()

In [8]:
# convo_df.head()

In [9]:
# utt_df = corpus.get_utterances_dataframe()

In [10]:
# utt_df.head()

In [11]:
# utt_df.to_csv('data/utterances.csv', index=False)
# speakers_df.to_csv('data/speakers.csv', index=False)
# convo_df.to_csv('data/conversations.csv', index=False)

## Pred DF

In [12]:
# utt_df = pd.read_csv("data/utterances.csv")

In [13]:
# pred_df = pd.DataFrame(columns=['utterance text', 'speaker', 'conversation_id', 'prediction'])
# pred_df['utterance text'] = utt_df['text']
# pred_df['speaker'] = utt_df['speaker']  
# pred_df['conversation_id'] = utt_df['conversation_id']

In [14]:
# pred_df = pred_df.dropna(subset=['utterance text'])

In [15]:
# pred_df.to_csv("data/predictions.csv")

## Predictions

In [16]:
pred_df = pd.read_csv("data/predictions.csv")

In [17]:
utt_text = np.array(pred_df['utterance text'])

In [18]:
# lengths = [len(s.split()) if type(s) == str else len(str(s).split()) for s in utt_text ]

In [19]:
# # Plot the histogram
# plt.hist(lengths, bins=range(1, max(lengths) +2 ) )
# plt.xlabel('String Length')
# plt.ylabel('Frequency')
# plt.title('Histogram of String Lengths')
# plt.show()

In [20]:
hyperparams = {
    'train_path':utt_text, 
    'backbone':'bert-base-uncased',
    'max_length': 128, #maximum context length for the model
    'batch_size': 128,
}

In [21]:
cornell_dataset = DataClass(hyperparams, utt_text, pred_mode=True)
cornell_data_loader = DataLoader(cornell_dataset,
                                    batch_size=hyperparams['batch_size'],
                                    shuffle=False
                                    )

/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...


/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter_2018 - 1grams ...


PreProcessing dataset ...:   0%|          | 0/304446 [00:00<?, ?it/s]/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
PreProcessing dataset ...:   0%|          | 1324/304446 [00:01<04:21, 1161.28it/s]

PreProcessing dataset ...:   6%|▌         | 18404/304446 [00:15<03:56, 1209.64it/s]

: 

In [ ]:
model_path = "../../model1/final_model.pt"
model = torch.load(model_path)

In [ ]:
raise Exception

Exception: 

In [ ]:
model.to(device)
model.eval()

index = 0

In [ ]:
with torch.no_grad():
    current_index = 0
    for step, batch in enumerate(progress_bar(cornell_data_loader, parent=None, leave=False)):
        if step < index:
            current_index += hyperparams['batch_size']
            continue
        _, num_rows, y_pred, logits, targets = model(batch, device)
        pred_df['prediction'][current_index: current_index + num_rows, :] = logits.cpu().numpy()
        if step % 10 == 0: pred_df.to_csv("data/predictions.csv")
        current_index += num_rows

pred_df.to_csv("data/predictions.csv")